In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as sk
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Dropout, LeakyReLU, Activation

from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# Load the dataset
df = pd.read_csv('ai4i2020.csv')
df['Machine functional'] = 1 - df['Machine failure']

# Display the first few lines of the dataset
print(df.head())

# First, understand the class distribution
print("\nClass distribution before rebalancing:")
print(df['Machine failure'].value_counts())
print("\nFailure types distribution:")
print("TWF:", df['TWF'].sum())
print("HDF:", df['HDF'].sum())
print("PWF:", df['PWF'].sum())
print("OSF:", df['OSF'].sum())

# Prepare features and target
X = df.drop(columns=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])
y = df[['Machine functional', 'TWF', 'HDF', 'PWF', 'OSF']]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE for oversampling (only on training data)
# We'll apply SMOTE separately for each target to ensure all classes are balanced
print("\nApplying SMOTE to balance all classes...")

# Create a synthetic dataset with more failure examples
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train['Machine functional'])
y_train_resampled = pd.DataFrame(y_train_resampled, columns=['Machine functional'])

# Now we need to associate the other failure types based on the Machine failure
for failure_type in ['TWF', 'HDF', 'PWF', 'OSF']:
    # For each newly created positive Machine failure case, randomly assign a failure type
    failure_indices = y_train_resampled[y_train_resampled['Machine failure'] == 1].index
    # Create a balanced distribution of failure types
    failure_count = len(failure_indices)
    type_count = failure_count // 4  # Divide equally between the 4 failure types
    remainder = failure_count % 4
    
    # Initialize the column with zeros
    y_train_resampled[failure_type] = 0
    
    # Distribute failure types
    current_idx = 0
    for i, failure_type_i in enumerate(['TWF', 'HDF', 'PWF', 'OSF']):
        count = type_count + (1 if i < remainder else 0)
        indices_to_assign = failure_indices[current_idx:current_idx+count]
        y_train_resampled.loc[indices_to_assign, failure_type_i] = 1
        current_idx += count

# Check the new class distribution
print("\nClass distribution after rebalancing (training set):")
print("Machine failure:", y_train_resampled['Machine failure'].value_counts())
print("TWF:", y_train_resampled['TWF'].sum())
print("HDF:", y_train_resampled['HDF'].sum())
print("PWF:", y_train_resampled['PWF'].sum())
print("OSF:", y_train_resampled['OSF'].sum())

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Create a model for multi-label classification
model = Sequential(
    [
        Dense(128, input_dim=X_train_scaled.shape[1]),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),

        Dense(64),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),
        
        Dense(32),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),

        Dense(5, activation='sigmoid')  # 5 outputs: Machine failure, TWF, HDF, PWF, OSF
    ]
)

model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train_scaled, 
    y_train_resampled, 
    epochs=20, 
    batch_size=32, 
    validation_split=0.2,
    class_weight={i: 1.0 for i in range(5)}  # Equal weight for all classes
)

# Evaluate the model
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# Display results for each target
target_names = ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF']
for i, target in enumerate(target_names):
    print(f"\nClassification report for {target}:")
    print(classification_report(y_test[target], y_pred[:, i]))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test[target], y_pred[:, i])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f'Not {target}', target])
    disp.plot()
    plt.title(f'Confusion Matrix for {target}')
    plt.show()

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

   UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860    M                298.1                    308.6   
1    2     L47181    L                298.2                    308.7   
2    3     L47182    L                298.1                    308.5   
3    4     L47183    L                298.2                    308.6   
4    5     L47184    L                298.2                    308.7   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  TWF  \
0                    1551         42.8                0                0    0   
1                    1408         46.3                3                0    0   
2                    1498         49.4                5                0    0   
3                    1433         39.5                7                0    0   
4                    1408         40.0                9                0    0   

   HDF  PWF  OSF  RNF  Machine functional  
0    0    0    0    0               

c:\Users\maure\OneDrive\Bureau\ISMIN\IA_manu_embarq\TP_IA_EMBARQUEE\myenv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


ValueError: could not convert string to float: 'L56434'